In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import os
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.formula.api as smf
import warnings
from sklearn.model_selection import cross_val_score
from datetime import datetime
from sklearn.cluster import MiniBatchKMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn import metrics
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from scipy.special import boxcox1p, inv_boxcox1p 

### Откроем файлы

In [2]:
# откроем файлы, которые мы сгенерировали ранее (на первой неделе)
all_data = []
for dic, folders, files in os.walk('/Users/shrlq/Documents/python_yandex/taxi_forecast/agg_data'):
    for file in files:
        print (file)
        data_month = pd.read_csv(str('/Users/shrlq/Documents/python_yandex/taxi_forecast/agg_data/' + file), ';', index_col=['tpep_pickup_hour'], 
                                 parse_dates=['tpep_pickup_hour'])
        all_data.append(data_month)
all_data = pd.concat(all_data)
all_data.sort_values(by = 'tpep_pickup_hour', inplace=True)
all_data = all_data.drop(['Unnamed: 0'], axis=1)
all_data.head()

df_03.csv
df_02.csv
df_06.csv
df_05.csv
df_04.csv


,Reg_ID,count
tpep_pickup_hour,,
2016-02-01,1841,0
2016-02-01,269,0
2016-02-01,988,0
2016-02-01,961,0
2016-02-01,668,0


In [3]:
# получим временные ряды по каждому региону
data_ = all_data.pivot_table(columns=['Reg_ID'], aggfunc=sum, values=['count'], index=all_data.index)
data_.columns = data_.columns.droplevel()
data_.head()
# таким образом, кол-во рядов = кол-во регионов, для которых делаем прогноз

Reg_ID,1,2,3,4,5,6,7,8,9,10,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
tpep_pickup_hour,,,,,,,,,,,,,,,,,,,,,
2016-02-01 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-02-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-02-01 02:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-02-01 03:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-02-01 04:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# исключим ряды, среднее которых за май меньше 5.
only_may = data_.iloc[2160:-720, :]
data_ = data_.iloc[:, np.where(only_may.mean(axis=0) >= 5)[0]]

### Напишем несколько функций, которые потом понадобятся

In [5]:
# функция для добавления признаков - гармоник Фурье
def sincos_feat_gen(data):
    q=6
    x = np.arange(1, data.shape[0])+2
    week_seas = []
    df = pd.DataFrame()
    for n in np.arange(1, q):
        week_sin = np.sin(x*2*np.pi*n/168)
        week_seas.append(week_sin)
        
        day_sin = np.sin(x*2*np.pi*n/24)
        week_seas.append(day_sin)
        
        week_cos = np.cos(x*2*np.pi*n/168)
        week_seas.append(week_cos)
        
        day_sin = np.sin(x*2*np.pi*n/24)
        week_seas.append(day_sin)
        
    df_= pd.DataFrame(data = week_seas).T
    sincos_col_names = ['sincos_' + str(x) for x in np.arange(0, len(np.arange(1, q))*4)]
    df_.columns = sincos_col_names
    return (df_)


In [6]:
# функция для добавления дискретного признака дня недели 
def wd_feat_gen(data, name='weekday'):   
    data_ = pd.DataFrame()
    data_[name] = [str(d.weekday()) for d in data.index]
    return(data_)

In [7]:
# функция для кодирования категорийных признаков в бинарные
def cat_feat_gen(data, columns):
    # добавим признак категориальные признаки дней недели:
    temp_data = data.loc[:, columns] 

    # с помощью OneHotEncoder закодируем признаки по дням неделям:
    encoder = OneHotEncoder(drop='first', categories='auto')
    data_oh = encoder.fit_transform(temp_data).toarray()
    data_oh = pd.DataFrame(data_oh)
    
    data_oh.index = data.index
    cat_names = ['cat_' + str(x) for x in np.arange(0, len(data_oh.columns))]
    data_oh.columns = cat_names
    return(data_oh)


In [8]:
# функция нахождения признака T+i в прошлом периоде с лагом -i
def count_i_lag(count, i):
    count_ = []
    for n in np.arange(count.shape[0]):
        if n+i >=0 and n+i < count.shape[0]:
            count_.append(count[n+i])
        else:
            count_.append(np.nan)
    return(count_)

In [9]:
# функция нахождения признака T+i в прошлом периоде с лагом -i (с накоплением)
def count_i_cumulative(count, i):
    count_ = []
    for n in np.arange(count.shape[0]):
        if n+i >=0 and n+i < count.shape[0]:
            count_.append(count[n+i:n].sum())
        else:
            count_.append(np.nan)
    return(count_)

In [10]:
# функция нахождения необходимых авторегрессионных признаков для временного ряда
def autoreg_feat_gen(data, column='count', 
                     p_=np.arange(0, 2+1, 1), # вектор, какие сколько часов назад брать (по умолчанию последние 2 часа)
                     pS_=np.arange(0, 24+1, 24), # вектор, какие сколько дней назад брать (по умолчанию последние 2 часа)
                     pCum_=np.arange(0, 3, 1)):
    # посчитаем признаки значения count в прошлом - count_P (равный T-p)
    count = data.loc[:, column]
    count_p=[]
    for p in p_:
        count_p.append(count_i_lag(count, i=-p))
    count_p = pd.DataFrame(count_p[1:])  # не считаем исходный ряд

    # посчитаем значения признака count в прошлом с сезонным лагом - count_Ps (равный T-p)
    count_pS=[]
    for p in pS_ :
        count_pS.append(count_i_lag(count, i=-p))
    count_pS = pd.DataFrame(count_pS[1:]) # не считаем исходный ряд

    # посчитаем накопленные значения признака count в прошлом - count_Pcum (равный T-p)
    count_pCum=[]
    for p in pCum_:
        count_pCum.append(count_i_cumulative(count, i=-p))
    count_pCum = pd.DataFrame(count_pCum[1:]) # не считаем исходный ряд

    # соберем новые полученные признаки в табличный вид
    past_col_names = ['past_' + str(x) for x in np.arange(0, len(p_)-1 + len(pS_)-1 + len(pCum_)-1)]
    past_df = pd.concat([count_p, count_pS, count_pCum]).T
    past_df.columns=past_col_names

    return(past_df)

In [11]:
# функция нахождения необходимых авторегрессионных признаков для каждого временного ряда
def feat_for_ts_gen_try(data,
                        p_=np.arange(0, 24+1, 1), # вектор признаков, где значения вектора - это кол-во поездок N часов назад
                        pS_=np.arange(0, 24*7+1, 24*7), # вектор признаков, где значения вектора - это кол-во поездок N часов назад (для сезонного лага)
                        pCum_=np.arange(0, 24+1, 24)): # вектор признаков, где значения вектора - это сумма поездок с N часов назад и по текущий час
    
    feat_df_list=[]
    for ind, reg in enumerate(data.columns):
        new_df = pd.DataFrame()
        new_df['tpep_pickup_hour'] = data.index
        new_df['Reg_ID'] = reg
        new_df['count'] = list(data[reg]) #добавили значение в текущий момент времени
        
        autoreg_feat_df = autoreg_feat_gen(data, 
                                           column=reg,
                                           p_=p_, 
                                           pS_=pS_, 
                                           pCum_=pCum_) # добавили авторегрессионные признаки

        sincos_feat_df = sincos_feat_gen(data) # добавили синусоидные и косинусоидные признаки
        
        feat_df = pd.concat([new_df, autoreg_feat_df, sincos_feat_df
                            ], axis=1, ignore_index=False)
        feat_df_list.append(feat_df)
    feat_data = pd.concat(feat_df_list) 
    return(feat_data)

In [12]:
# функция для стандартизации выбранных признаков
def standardize_only_num_feat (data_train, data_predict, arg=None):
    SC = StandardScaler()
    model = SC.fit(data_train)
    data_pr = SC.transform(data_predict)
    data_pr = pd.DataFrame(data_pr)
    return (data_pr)

### Сгенерируем различные признаки сезонного и авторегрессионного характера

In [13]:
# Построим признаки, помогающие строить регрессию для временных рядов
feat_data = feat_for_ts_gen_try(data_,
                               p_=np.arange(0, 24+1, 1), # возьмем поездки за каждый из последних 24 часов
                               pS_=np.arange(0, 24*7+1, 24*7), # возьмем поездки в аналогичный час за каждый из последних 7 дней
                               pCum_=np.arange(0, 24+1, 24)) # возьмем суммарное кол-во поездок за последние сутки

# Удалим строки с пустыми значениями
feat_data_ = feat_data.dropna(axis=0)
feat_data_.reset_index(inplace=True, drop=True)
print ('Удалено значений:' + str(feat_data.shape[0] - feat_data_.shape[0]))

feat_data_.head()

Удалено значений:17238


,tpep_pickup_hour,Reg_ID,count,past_0,past_1,past_2,past_3,past_4,past_5,past_6,...,sincos_10,sincos_11,sincos_12,sincos_13,sincos_14,sincos_15,sincos_16,sincos_17,sincos_18,sincos_19
0,2016-02-08 00:00:00,1075,24,59.0,49.0,25.0,29.0,23.0,54.0,63.0,...,0.943883,7.071068e-01,0.433884,1.763863e-14,0.900969,1.763863e-14,0.532032,-0.707107,0.846724,-0.707107
1,2016-02-08 01:00:00,1075,9,24.0,59.0,49.0,25.0,29.0,23.0,54.0,...,0.900969,8.818695e-15,0.563320,-8.660254e-01,0.826239,-8.660254e-01,0.680173,-0.866025,0.733052,-0.866025
2,2016-02-08 02:00:00,1075,4,9.0,24.0,59.0,49.0,25.0,29.0,23.0,...,0.846724,-7.071068e-01,0.680173,-8.660254e-01,0.733052,-8.660254e-01,0.804598,0.258819,0.593820,0.258819
3,2016-02-08 03:00:00,1075,1,4.0,9.0,24.0,59.0,49.0,25.0,29.0,...,0.781831,-1.000000e+00,0.781831,-2.841923e-14,0.623490,-2.841923e-14,0.900969,1.000000,0.433884,1.000000
4,2016-02-08 04:00:00,1075,4,1.0,4.0,9.0,24.0,59.0,49.0,25.0,...,0.707107,-7.071068e-01,0.866025,8.660254e-01,0.500000,8.660254e-01,0.965926,0.258819,0.258819,0.258819


#### Задание 1
У нас 6 задач прогнозирования (на 1 ... 6 часов вперед). 
Для каждой из этих задач сформируем выборки и свою целевую переменную.


In [14]:
# посчитаем для каждой из 6 задач прогнозирования целевые значения count_I (T+i)
# получим на выходе таблицу с 6 колонками - целевыми значениями
targets=[]
for i in list(range(6)):
    target_ = count_i_lag(feat_data_.loc[:, 'count'], i=i+1)
    targets.append(target_)

targ_df = pd.DataFrame(targets, index=['targ_' + str(i+1) for i in list(range(6))]).T
targ_feat_data = pd.concat([targ_df, feat_data_], axis=1) #объединим с признаками 
targ_feat_data.head()

,targ_1,targ_2,targ_3,targ_4,targ_5,targ_6,tpep_pickup_hour,Reg_ID,count,past_0,...,sincos_10,sincos_11,sincos_12,sincos_13,sincos_14,sincos_15,sincos_16,sincos_17,sincos_18,sincos_19
0,9.0,4.0,1.0,4.0,11.0,28.0,2016-02-08 00:00:00,1075,24,59.0,...,0.943883,7.071068e-01,0.433884,1.763863e-14,0.900969,1.763863e-14,0.532032,-0.707107,0.846724,-0.707107
1,4.0,1.0,4.0,11.0,28.0,45.0,2016-02-08 01:00:00,1075,9,24.0,...,0.900969,8.818695e-15,0.563320,-8.660254e-01,0.826239,-8.660254e-01,0.680173,-0.866025,0.733052,-0.866025
2,1.0,4.0,11.0,28.0,45.0,62.0,2016-02-08 02:00:00,1075,4,9.0,...,0.846724,-7.071068e-01,0.680173,-8.660254e-01,0.733052,-8.660254e-01,0.804598,0.258819,0.593820,0.258819
3,4.0,11.0,28.0,45.0,62.0,70.0,2016-02-08 03:00:00,1075,1,4.0,...,0.781831,-1.000000e+00,0.781831,-2.841923e-14,0.623490,-2.841923e-14,0.900969,1.000000,0.433884,1.000000
4,11.0,28.0,45.0,62.0,70.0,50.0,2016-02-08 04:00:00,1075,4,1.0,...,0.707107,-7.071068e-01,0.866025,8.660254e-01,0.500000,8.660254e-01,0.965926,0.258819,0.258819,0.258819


In [15]:
# сохраним в отдельный файл результат
targ_feat_data.to_csv('targ_feat_data.csv', ';', 
                      index=False)

targ_feat_data.head()

,targ_1,targ_2,targ_3,targ_4,targ_5,targ_6,tpep_pickup_hour,Reg_ID,count,past_0,...,sincos_10,sincos_11,sincos_12,sincos_13,sincos_14,sincos_15,sincos_16,sincos_17,sincos_18,sincos_19
0,9.0,4.0,1.0,4.0,11.0,28.0,2016-02-08 00:00:00,1075,24,59.0,...,0.943883,7.071068e-01,0.433884,1.763863e-14,0.900969,1.763863e-14,0.532032,-0.707107,0.846724,-0.707107
1,4.0,1.0,4.0,11.0,28.0,45.0,2016-02-08 01:00:00,1075,9,24.0,...,0.900969,8.818695e-15,0.563320,-8.660254e-01,0.826239,-8.660254e-01,0.680173,-0.866025,0.733052,-0.866025
2,1.0,4.0,11.0,28.0,45.0,62.0,2016-02-08 02:00:00,1075,4,9.0,...,0.846724,-7.071068e-01,0.680173,-8.660254e-01,0.733052,-8.660254e-01,0.804598,0.258819,0.593820,0.258819
3,4.0,11.0,28.0,45.0,62.0,70.0,2016-02-08 03:00:00,1075,1,4.0,...,0.781831,-1.000000e+00,0.781831,-2.841923e-14,0.623490,-2.841923e-14,0.900969,1.000000,0.433884,1.000000
4,11.0,28.0,45.0,62.0,70.0,50.0,2016-02-08 04:00:00,1075,4,1.0,...,0.707107,-7.071068e-01,0.866025,8.660254e-01,0.500000,8.660254e-01,0.965926,0.258819,0.258819,0.258819


### Подготовим датасеты для обучения и тестирования

#### Задание 2
Разобьем каждую выборку на три части: обучающую (до мая), тестовую (май) и валидационную (июнь)

In [16]:
# разобьем выборки на обучающую, тестовую и итоговую по датам конца истории
may_start = datetime.strptime('2016-04-30 23:00:00', '%Y-%m-%d %H:%M:%S')
may_end = datetime.strptime('2016-05-31 17:00:00', '%Y-%m-%d %H:%M:%S')

june_start = datetime.strptime('2016-05-31 23:00:00', '%Y-%m-%d %H:%M:%S')
june_end = datetime.strptime('2016-06-30 17:00:00', '%Y-%m-%d %H:%M:%S')

train_cond = np.where(targ_feat_data['tpep_pickup_hour'] <= may_start)[0]
data_train = targ_feat_data.iloc[train_cond, 8:] 
y_train = targ_feat_data.iloc[train_cond, :6]    

test_cond = np.where((targ_feat_data['tpep_pickup_hour'] >= may_start) & (targ_feat_data['tpep_pickup_hour'] <= may_end))[0]
data_test = targ_feat_data.iloc[test_cond, 8:]
y_test = targ_feat_data.iloc[test_cond, :6] 

val_cond = np.where((targ_feat_data['tpep_pickup_hour'] >= june_start) & (targ_feat_data['tpep_pickup_hour'] <= june_end))[0]
data_val_ = targ_feat_data.iloc[val_cond, 8:]
y_val = targ_feat_data.iloc[val_cond, :6]

print ('Кол-во строк в обуч. выборке: ' + str(data_train.shape[0]))
print ('Кол-во строк в тест. выборке: ' + str(data_test.shape[0]))
print ('Кол-во строк в валид. выборке: ' + str(data_val_.shape[0]))

Кол-во строк в обуч. выборке: 203184
Кол-во строк в тест. выборке: 75378
Кол-во строк в валид. выборке: 72930


#### Задание 3
Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016

In [17]:
from sklearn import ensemble

metric_train = []
metric_test = []
a = []
b = []

for i in [100]: # [1500, 2500 ...] здесь был вектор с параметрами n_estimators, но для быстроты возьмем только одно значение
    for ind in range(6):
        RF = ensemble.RandomForestRegressor(n_estimators=i, n_jobs=10)
        RF.fit(data_train, y_train.iloc[:, ind])
        pred_train = RF.predict(data_train)
        pred_test = RF.predict(data_test)
        metric_train.append(metrics.mean_absolute_error(pred_train, y_train.iloc[:, ind]))
        metric_test.append(metrics.mean_absolute_error(pred_test, y_test.iloc[:, ind]))
        print(str(i) + ' ' + str(ind)) 
    a.append(np.mean(metric_train))
    b.append(np.mean(metric_test))

100 0
100 1
100 2
100 3
100 4
100 5


In [24]:
print('Средняя ошибка на обучающей выборке: ' + str(a))
print('Средняя ошибка на тестовой выборке: ' + str(b))

Средняя ошибка на обучающей выборке: [6.5851507172480765]
Средняя ошибка на тестовой выборке: [19.59654903729647]


Данный гиперпараметр применен для примера и в качестве увеличения скорости расчетов (лучший результат получается при n = 3000)

### Сделаем прогнозы на май и посчитаем ошибку

#### Задание 4
Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд. Посчитайте в ноутбуке ошибку прогноза.

In [30]:
# для каждого ряда посчитаем модель на основе данных по апрель 2016 года
# построим прогноз с мая 2016 на 6 часов для каждого конца истории, начиная с 30 апр 23:00
R=102
step = 6
predictions_may = []
predictions_jun = []

for ind in list(range(step)):
    RF = ensemble.RandomForestRegressor(n_estimators=100, n_jobs=10)
    model = RF.fit(X = data_train, y = y_train.iloc[:, ind])
    print('train: ' + str(ind))
    
    # построим прогнозы на май с плавающим концом истории и запишем прогнозы:
    day_in_period = 31
    firt_day_ind = 90
    for y in np.arange(0, day_in_period*24-step+1): # должно совпадать с кол-вом строк в тест, выборке
        prediction = model.predict(data_test.iloc[y+np.arange(0, R*(day_in_period*24-step+1), day_in_period*24-step+1), :])
        pred_reg_df = pd.DataFrame(prediction, columns=['prediction'])
        pred_reg_df['target'] = list(y_test.iloc[y+np.arange(0, R*(day_in_period*24-step+1), day_in_period*24-step+1), ind]) # прогнозируем для 1 конца истории по всем рядам
        pred_reg_df['geoID'] = data_.columns[:R] # для всех рядов одинаковый конец истории, нач. с 31 апр 23ч
        pred_reg_df['histEndDay'] = str(datetime.strftime(data_.index[firt_day_ind*24-1+y], '%Y-%m-%d'))
        pred_reg_df['histEndHour'] = int(datetime.strftime(data_.index[firt_day_ind*24-1+y], '%H'))
        pred_reg_df['step'] = ind+1
        predictions_may.append(pred_reg_df)
    print('predict_may: ' + str(ind))
    
    # построим прогнозы на июнь с плавающим концом истории и запишем прогнозы:
    day_in_period = 30
    firt_day_ind = 90+31
    for y in np.arange(0, day_in_period*24-step+1): # должно совпадать с кол-вом строк в тест, выборке
        prediction = model.predict(data_val_.iloc[y+np.arange(0, R*(day_in_period*24-step+1), day_in_period*24-step+1), :])
        pred_reg_df = pd.DataFrame(prediction, columns=['prediction'])
        pred_reg_df['geoID'] = data_.columns[:R] # для всех рядов одинаковый конец истории, нач. с 31 мая 23ч
        pred_reg_df['histEndDay'] = str(datetime.strftime(data_.index[firt_day_ind*24-1+y], '%Y-%m-%d'))
        pred_reg_df['histEndHour'] = int(datetime.strftime(data_.index[firt_day_ind*24-1+y], '%H'))
        pred_reg_df['step'] = ind+1
        predictions_jun.append(pred_reg_df)
    print('predict_jun: ' + str(ind))

pred_data_may = pd.concat(predictions_may, axis=0, ignore_index=True)
pred_data_jun = pd.concat(predictions_jun, axis=0, ignore_index=True)

pred_data_may.head()

train: 0
predict_may: 0
predict_jun: 0
train: 1
predict_may: 1
predict_jun: 1
train: 2
predict_may: 2
predict_jun: 2
train: 3
predict_may: 3
predict_jun: 3
train: 4
predict_may: 4
predict_jun: 4
train: 5
predict_may: 5
predict_jun: 5


,prediction,target,geoID,histEndDay,histEndHour,step
0,70.14,71.0,1075,2016-04-30,23,1
1,82.70,64.0,1076,2016-04-30,23,1
2,48.77,52.0,1077,2016-04-30,23,1
3,86.17,81.0,1125,2016-04-30,23,1
4,251.95,259.0,1126,2016-04-30,23,1


In [31]:
# посчитаем ошибку (именно формула этой ошибки указана в задании)
pred_data_may['resid'] = np.absolute(pred_data_may.prediction - pred_data_may.target)
MAE_simp_reg = pred_data_may[['resid']].mean(axis=0)
MAE_simp_reg
pred_data_may['resid'].describe()

count    452268.000000
mean         19.594649
std          38.120103
min           0.000000
25%           2.220000
50%           6.140000
75%          19.860000
max         974.720000
Name: resid, dtype: float64

In [42]:
#запишем предсказания за май в удобный формат
id_column = [str(x['geoID'])+'_'+str(x['histEndDay'])+'_'+str(x['histEndHour'])+'_'+str(x['step']) 
             for i, x in pred_data_may.iterrows()]
y_column = pred_data_may['prediction']
target_column = pred_data_may['target']

pred_data_may_format = pd.DataFrame([id_column,y_column,target_column], index=['id', 'y_RF', 'target']).T
pred_data_may_format.to_csv('/Users/shrlq/Documents/python_yandex/taxi_forecast/agg_data/predictions_may_RF.csv', index=False)
pred_data_may_format.head()

,id,y_RF,target
0,1075_2016-04-30_23_1,70.14,71
1,1076_2016-04-30_23_1,82.7,64
2,1077_2016-04-30_23_1,48.77,52
3,1125_2016-04-30_23_1,86.17,81
4,1126_2016-04-30_23_1,251.95,259


In [34]:
#запишем предсказания за июнь в удобный формат
id_column = [str(x['geoID'])+'_'+str(x['histEndDay'])+'_'+str(x['histEndHour'])+'_'+str(x['step']) 
             for i, x in pred_data_jun.iterrows()]
y_column = pred_data_jun['prediction']

RF_jun = pd.DataFrame([id_column,y_column], index=['id', 'y_RF']).T
RF_jun.to_csv('/Users/shrlq/Documents/python_yandex/taxi_forecast/agg_data/predictions_jun_RF.csv', index=False)
RF_jun.head()

,id,y_RF
0,1075_2016-05-31_23_1,18.52
1,1076_2016-05-31_23_1,37.65
2,1077_2016-05-31_23_1,16.96
3,1125_2016-05-31_23_1,21.19
4,1126_2016-05-31_23_1,67.28


### Добавим признак с прошлой недели (предсказания SARIMAX)

In [39]:
# возьмем предсказания Аримы
sarimax = pd.read_csv('/Users/shrlq/Documents/python_yandex/course 6/week_4/predictions_test.csv',
                     parse_dates=['tpep_pickup_hour'])
# приведем таблицу c 4-й недели к единому формату
from datetime import datetime
sarimax_melt = sarimax.melt(id_vars=['Reg_ID', 'tpep_pickup_hour'])
id_ = sarimax_melt['Reg_ID'].astype(str)
date_ = sarimax_melt['tpep_pickup_hour'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d')).astype(str)
hour_ = sarimax_melt['tpep_pickup_hour'].apply(lambda x: datetime.strftime(x, '%H')).astype(int).astype(str)
step_ = sarimax_melt['variable'].astype(str)
y_ = sarimax_melt['value']

sarimax = pd.DataFrame()
sarimax['id'] = id_ + '_' + date_ + '_' + hour_ + '_' + step_
sarimax['y_sarimax'] = y_
sarimax.head()

,id,y_sarimax
0,1075_2016-04-30_23_1,69.653230
1,1075_2016-05-01_0_1,51.913017
2,1075_2016-05-01_1_1,30.438599
3,1075_2016-05-01_2_1,12.713545
4,1075_2016-05-01_3_1,21.911447


In [43]:
# объеденим предсказания и будем использовать их в качестве признаков для линейной регрессии
all_predictions_test = pred_data_may_format.merge(sarimax, 
                                            on=['id'])
all_predictions_test.head()

,id,y_RF,target,y_sarimax
0,1075_2016-04-30_23_1,70.14,71,69.653230
1,1076_2016-04-30_23_1,82.7,64,101.754040
2,1077_2016-04-30_23_1,48.77,52,56.291121
3,1125_2016-04-30_23_1,86.17,81,82.435531
4,1126_2016-04-30_23_1,251.95,259,206.944234


### Построим еще одну модель - Ridge регрессию
Ridge модель будет использовать в качестве признаков предсказания за май с этой (RF) и предыдущей (SARIMAX) недель.

In [47]:
# модель будем обучать на данных за май и ошибку будем оценивать на тех же данных с помощью кросс-валидации
from sklearn.linear_model import Ridge
RR = Ridge(alpha=.00001, fit_intercept=True, normalize=True,
           max_iter=None, tol=0.001, random_state=122)
model_selection.cross_validate(RR, X=all_predictions_test[['y_RF', 'y_sarimax']], 
                              y=all_predictions_test[['target']], 
                              cv=6, scoring='neg_mean_absolute_error',
                             n_jobs=-1)

{'fit_time': array([0.10657501, 0.09422231, 0.1098249 , 0.12574697, 0.11235499,
        0.08908701]),
 'score_time': array([0.01211882, 0.01095104, 0.01600504, 0.0144341 , 0.01291299,
        0.01089096]),
 'test_score': array([-15.47988823, -18.68934294, -19.90062124, -20.69658913,
        -21.10118593, -21.28999221])}

In [48]:
# обучим модель
RR_model = RR.fit(X=all_predictions_test[['y_RF', 'y_sarimax']], 
                              y=all_predictions_test[['target']])
print (RR_model.coef_, RR_model.intercept_)

[[0.82697035 0.1586442 ]] [0.68360429]


Полученные к-ты можно интерпретировать как веса моделей в зависимости от того, какая лучше предсказывает. Модель RF имеет значительно больший вес (.83 по сравнению с .16 у ARIMA). 

#### Посмотрим, как улучшилась ошибка

In [49]:
test_er_df = pd.DataFrame()
test_er_df['rr'] = (RR.predict(X=all_predictions_test[['y_RF', 'y_sarimax']])).reshape(all_predictions_test.shape[0],
                                                                                      )
test_er_df['y'] = all_predictions_test['target']
test_er_df['resid'] = np.absolute(test_er_df.rr - test_er_df.y)
test_er_df['resid'] = test_er_df['resid'].astype(float)
simp_reg = test_er_df[['resid']].mean(axis=0)

test_er_df['resid'].describe()

count    452268.000000
mean         19.518161
std          36.919857
min           0.000016
25%           2.376112
50%           6.264293
75%          20.172847
max         919.015992
Name: resid, dtype: float64

Ошибка незначительно улучшилась на сотые доли кол-ва поездок.

### Задание 5
Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 

In [58]:
# возьмем предсказания Аримы за июнь
sarimax_jun = pd.read_csv('/Users/shrlq/Documents/python_yandex/course 6/week_4/predictions_jun_submit.csv')
sarimax_jun.rename(columns = {'y': 'y_sarimax',
                             'Reg_ID': 'id'}, inplace=True)
sarimax_jun.head()

,id,y_sarimax
0,1075_2016-05-31_23_1,23.882802
1,1075_2016-06-01_0_1,18.235073
2,1075_2016-06-01_1_1,1.179826
3,1075_2016-06-01_2_1,0.751935
4,1075_2016-06-01_3_1,-0.173357


In [59]:
# сформируем датасет, где признаками являются полученные прознозы на июнь в RF и SARIMAX
all_predictions = RF_jun.merge(sarimax_jun, on=['id'])
all_predictions.head()

,id,y_RF,y_sarimax
0,1075_2016-05-31_23_1,18.52,23.882802
1,1076_2016-05-31_23_1,37.65,39.365908
2,1077_2016-05-31_23_1,16.96,9.215687
3,1125_2016-05-31_23_1,21.19,30.845351
4,1126_2016-05-31_23_1,67.28,68.851299


#### Спрогнозируем данные на июнь с помощью Ridge регрессии

In [60]:
col_y = pd.DataFrame(RR.predict(all_predictions.drop(['id'], axis=1)), columns=['y'])
col_id = all_predictions['id']

predictions_jun_submit = pd.DataFrame()
predictions_jun_submit = pd.concat([col_id,col_y], axis=1)

# запишем результат в файл
# predictions_jun_submit.to_csv('predictions_jun_submit.csv', index=False)